Libraries

In [1]:
import numpy as np
import pandas as pd
from itertools import combinations
from qiskit.quantum_info import SparsePauliOp, Pauli, state_fidelity, Statevector
from scipy.linalg import expm
from qiskit import QuantumCircuit
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import SuzukiTrotter
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from qiskit_aer import AerSimulator

Global parameters

In [2]:
N_QUBITS = 5
N = 2**N_QUBITS
TAU = 30e-9              # 30 ns (seconds)
A0 = 2 * np.pi * 15e9       # rad/s
B0 = 2 * np.pi * 11e9       # rad/s
REPS = 1
M = 1
#REPS = 64
#M = int(np.ceil(2000 / REPS))    # -> 32 slices when REPS=64
DELTA_T = TAU / M
ORDER = 2                # Suzuki-Trotter order
RANDOM_STATE = 42
N_SPLITS = 5

# added by Jiri
OPTIMIZATION_LEVEL = 0
SHOTS = 1024
NOISE_MODEL = None

Dataset

In [3]:
dataset_save = "Dataset/Toxicity-13F.csv"
dataset_5F = "Dataset/Toxicity-5F.csv"
dataset_8F = "Dataset/Toxicity-8F.csv"
    
df = pd.read_csv(dataset_save)
df['Class'] = df['Class'].apply(lambda v: 1 if str(v).strip().lower().startswith('non') else 0)

df5 = pd.read_csv(dataset_5F)

df8 = pd.read_csv(dataset_8F)



Building model

In [4]:
def s_of_t(t, tau=TAU):
    # s(t) = sin^2( (pi/2) * sin^2(pi t / (2 tau)) )
    inner = np.sin(np.pi * t / (2 * tau)) ** 2
    return np.sin((np.pi / 2) * inner) ** 2

def A_of_t(t): 
    return A0 * (1.0 - s_of_t(t))

def B_of_t(t): 
    return B0 * s_of_t(t)

def build_HD(n):
    #H_D = - sum_i X_i represented as a SparsePauliOp
    labels = []
    coeffs = []
    for i in range(n):
        s = ['I'] * n
        s[i] = 'X'
        labels.append(''.join(s))
        coeffs.append(-1.0)
    pauli_list = [Pauli(label) for label in labels]
    return SparsePauliOp(pauli_list, coeffs)

def build_HD_matrix(n):
    # Single-qubit Pauli-X
    X = np.array([[0, 1],
                  [1, 0]], dtype=complex)
    I = np.eye(2, dtype=complex)

    H = np.zeros((2**n, 2**n), dtype=complex)
    for i in range(n):
        # Start from identity and apply X on qubit i
        ops = []
        for j in range(n):
            ops.append(X if j == i else I)
        # Tensor product (Kronecker product) to get full operator for this i
        H_i = ops[0]
        for op in ops[1:]:
            H_i = np.kron(H_i, op)
        H -= H_i  # sum_i (-X_i)
    return H

def build_HP_from_sample(x_std, Jij):
    # Problem Hamiltonian HP(x) = sum_i hi Z_i + sum_{i<j} Jij Z_i Z_j
    # where hi = x_i and Jij is correlation matrix entries.
    n = len(x_std)
    labels = []
    coeffs = []
    # local fields
    for i in range(n):
        s = ['I'] * n
        s[i] = 'Z'
        labels.append(''.join(s))
        coeffs.append(float(x_std[i]))
    # pairwise ZZ
    for i, j in combinations(range(n), 2):
        s = ['I'] * n
        s[i] = 'Z'
        s[j] = 'Z'
        labels.append(''.join(s))
        coeffs.append(float(Jij[i, j]))
    pauli_list = [Pauli(label) for label in labels]
    return SparsePauliOp(pauli_list, coeffs)

def build_HP_matrix_diagonal(x_std, Jij):
    """
    Build H_P as a dense matrix by computing its diagonal (since H_P is diagonal).
    Returns a (2**n, 2**n) ndarray.

    Conventions:
    - x_std: shape (n,) local fields h_i
    - Jij: (n,n) symmetric coupling matrix with zeros on diagonal
    - Qubit ordering: qubit 0 is the most-significant bit (MSB) in basis index.
    """
    x = np.asarray(x_std).ravel()
    n = x.shape[0]
    dim = 1 << n  # 2**n

    # Precompute z_i for each basis state and each qubit by bit operations
    # We will compute diag[k] = sum_i h_i * z_i(k) + sum_{i<j} Jij[i,j] * z_i(k) * z_j(k)
    # where z_i(k) = +1 if bit i == 0 else -1. With the MSB convention:
    # bit for qubit i is ((k >> (n-1-i)) & 1)
    diag = np.zeros(dim, dtype=float)
    indices = np.arange(dim, dtype=int)

    # Precompute z matrix maybe expensive memory-wise for large n; instead compute per qubit
    # but we can compute z_i for all indices quickly:
    z_bits = np.zeros((n, dim), dtype=int)
    for i in range(n):
        bitpos = n - 1 - i
        z_bits[i] = 1 - 2 * ((indices >> bitpos) & 1)  # +1 or -1

    # local fields contribution
    # diag += sum_i h_i * z_i
    diag += np.dot(x, z_bits)  # shape (dim,)

    # pairwise contribution: sum_{i<j} Jij[i,j] * z_i * z_j
    # We can compute this pairwise loop (n small => cheap)
    for i, j in combinations(range(n), 2):
        if Jij[i, j] == 0:
            continue
        diag += Jij[i, j] * (z_bits[i] * z_bits[j])

    # return full dense matrix (diagonal)
    return np.diag(diag)

def make_evolution_circuit(n_qubits, HP_op_time_dep_fn):
    qc = QuantumCircuit(n_qubits)
    # initial |+>^{n}
    for q in range(n_qubits):
        qc.h(q)

    # Prebuild H_D (Pauli sum op)
    HD_op = build_HD(n_qubits)

    # Append m slices. For slice k (0..m-1) freeze at midpoint t_{k+1/2} = (k+0.5)*dt
    for k in range(M):
        t_mid = (k + 0.5) * DELTA_T
        A = A_of_t(t_mid)
        B = B_of_t(t_mid)
        HP_mid = HP_op_time_dep_fn(k)

        # H_slice = A * HD + B * HP_mid
        H_slice = (A * HD_op) + (B * HP_mid)

        evo_gate = PauliEvolutionGate(H_slice, time=DELTA_T, synthesis=SuzukiTrotter(order=ORDER, reps=REPS))
        qc.append(evo_gate, qc.qubits)

    return qc

def build_circuit_for_sample(x_std, Jij):
    n = len(x_std)
    HP_op = build_HP_from_sample(x_std, Jij)
    qc = make_evolution_circuit(n, lambda k: HP_op)
    return qc

Exact solution

In [5]:
def exact_time_evolution_state(x_std, Jij):
    n = N_QUBITS
    #HD = build_HD(n)
    HD = build_HD_matrix(n)

    def evolve_one_sample(x_sample):
        # x_sample must be 1D length n
        dim = 2 ** n
        psi = np.ones((dim,), dtype=complex) / np.sqrt(dim)
        for k in range(M):
            t_mid = (k + 0.5) * DELTA_T
            A = A_of_t(t_mid)
            B = B_of_t(t_mid)
            #HP = build_HP_from_sample(x_sample, Jij)
            HP = build_HP_matrix_diagonal(x_sample, Jij)
            H_slice = (A * HD) + (B * HP)
            #H_mat = H_slice.to_matrix()
            U = expm(-1j * H_slice * DELTA_T)
            psi = U.dot(psi)
        
        return Statevector(psi)
    psis = []
    for i in range(len(x_std)):
        x_sample = x_std[i]
        psi = evolve_one_sample(x_sample)
        psis.append(psi)

    return psis

def comparison_of_states(psi1, psi2):
    fidelity = state_fidelity(psi1, psi2, validate=True)
    return fidelity

Approximation (using Trotterization)

In [17]:
def make_evolution_for_sample(n_qubits, HP_op_time_dep_fn):
    qc = QuantumCircuit(n_qubits)
    # initial |+>^{n}
    for q in range(n_qubits):
        qc.h(q)

    # Prebuild H_D (Pauli sum op)
    HD_op = build_HD(n_qubits)

    # Append m slices. For slice k (0..m-1) freeze at midpoint t_{k+1/2} = (k+0.5)*dt
    for k in range(M):
        t_mid = (k + 0.5) * DELTA_T
        A = A_of_t(t_mid)
        B = B_of_t(t_mid)
        HP_mid = HP_op_time_dep_fn(k)

        # H_slice = A * HD + B * HP_mid
        H_slice = (A * HD_op) + (B * HP_mid)

        evo_gate = PauliEvolutionGate(H_slice, time=DELTA_T, synthesis=SuzukiTrotter(order=ORDER, reps=REPS))
        qc.append(evo_gate, qc.qubits)
        statevector = Statevector.from_instruction(qc)
        #simulator = AerSimulator(method='DensityMatrix')

    return statevector.data

def build_circuit_for_sample(x_std, Jij):
    n = len(x_std)
    HP_op = build_HP_from_sample(x_std, Jij)
    data = make_evolution_for_sample(n, lambda k: HP_op)
    return data

def evolve_all(x_std, Jij):
    n_samples = len(x_std)
    all_data = []
    for i in range(n_samples):
        x_sample = x_std[i]
        data = build_circuit_for_sample(x_sample, Jij)
        all_data.append(data)
    return all_data



Comparison

In [14]:
X = df5.drop(columns=['Class']).values
y = df5['Class'].values

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

fold_idx = 0 
for train_idx, test_idx in skf.split(X, y): 
    fold_idx += 1 
    Xtrain, Xtest = X[train_idx], X[test_idx] 
    ytrain, ytest = y[train_idx], y[test_idx]

    # preprocessing 
    imputer = SimpleImputer(strategy='median') 
    imputer.fit(Xtrain) 
    Xtrain_im = imputer.transform(Xtrain) 
    Xtest_im = imputer.transform(Xtest) 

    scaler = StandardScaler() 
    scaler.fit(Xtrain_im) 
    Xtrain_std = scaler.transform(Xtrain_im) 
    Xtest_std = scaler.transform(Xtest_im) 

    # Jij computed from training set Pearson correlations, diag set to 0 
    rho = np.corrcoef(Xtrain_std, rowvar=False) 
    np.fill_diagonal(rho, 0.0) 
    Jij = rho.copy() 

    psi_exact = exact_time_evolution_state(Xtrain_std, Jij)
    psi_trotter = evolve_all(Xtrain_std, Jij)
    
    fidelities = []
    for i in range(len(psi_exact)):
        fid = comparison_of_states(psi_exact[i], psi_trotter[i])
        fidelities.append(fid)

    print(np.mean(fidelities))
    print("--------------")

c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


0.9999999999999991
--------------


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


0.9999999999999992
--------------


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


0.9999999999999992
--------------


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


0.9999999999999992
--------------


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\_index.py:168: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil and dok are more efficient.
  self._set_intXint(row, col, x.flat[0])


0.9999999999999992
--------------


In [18]:
import numpy as np
from qiskit.quantum_info import Operator
from scipy.linalg import expm

X = df5.drop(columns=['Class']).values
y = df5['Class'].values

skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE)

#TAU = 30e-9              # 30 ns (seconds)
#A0 = 2 * np.pi * 15e9       # rad/s
#B0 = 2 * np.pi * 11e9 
TAU = 100
A0 = 2 * np.pi
B0 = 2 * np.pi   
REPS = 50
M = 4

fold_idx = 0 
for train_idx, test_idx in skf.split(X, y): 
    fold_idx += 1 
    Xtrain, Xtest = X[train_idx], X[test_idx] 
    ytrain, ytest = y[train_idx], y[test_idx]

    # preprocessing 
    imputer = SimpleImputer(strategy='median') 
    imputer.fit(Xtrain) 
    Xtrain_im = imputer.transform(Xtrain) 
    Xtest_im = imputer.transform(Xtest) 

    scaler = StandardScaler() 
    scaler.fit(Xtrain_im) 
    Xtrain_std = scaler.transform(Xtrain_im) 
    Xtest_std = scaler.transform(Xtest_im) 

    # Jij computed from training set Pearson correlations, diag set to 0 
    rho = np.corrcoef(Xtrain_std, rowvar=False) 
    np.fill_diagonal(rho, 0.0) 
    Jij = rho.copy()
    # pick one sample
    i = 0
    x_sample = Xtrain_std[i]         # or Xtrain_std[0]
    n = len(x_sample)

    # Build the dense HP diagonal and HD dense used by exact evolution:
    HP_dense = build_HP_matrix_diagonal(x_sample, Jij)     # (2**n,2**n)
    HD_dense = build_HD_matrix(n)

    # Compute dense H_slice used in exact method (for k=0 slice)
    t_mid = (0 + 0.5) * DELTA_T
    A = A_of_t(t_mid)
    B = B_of_t(t_mid)
    H_slice_dense = (A * HD_dense) + (B * HP_dense)

    # Now build the SparsePauliOp version used in circuit path:
    HP_sparse = build_HP_from_sample(x_sample, Jij)   # SparsePauliOp
    HD_sparse = build_HD(n)                           # SparsePauliOp

    # Convert sparse ops to dense matrices and compare
    HP_sparse_mat = HP_sparse.to_matrix()             # dense
    HD_sparse_mat = HD_sparse.to_matrix()

    print("|| HP_sparse - HP_dense ||_inf =", np.max(np.abs(HP_sparse_mat - HP_dense)))
    print("|| HD_sparse - HD_dense ||_inf =", np.max(np.abs(HD_sparse_mat - HD_dense)))

    # build H_slice from SparsePauliOp (matrix)
    H_slice_from_sparse = A * HD_sparse_mat + B * HP_sparse_mat

    print("|| H_slice_dense - H_slice_from_sparse ||_inf =", np.max(np.abs(H_slice_dense - H_slice_from_sparse)))

    # Build the PauliEvolutionGate object (same as in your circuit)
    H_slice_opflow = (A * build_HD(n)) + (B * build_HP_from_sample(x_sample, Jij))
    evo_gate = PauliEvolutionGate(H_slice_opflow, time=DELTA_T,
                                synthesis=SuzukiTrotter(order=ORDER, reps=REPS))

    # Convert evo_gate to an Operator (unitary) by appending to 1-qubit-per-system circuit
    from qiskit import QuantumCircuit
    qc_test = QuantumCircuit(n)
    qc_test.append(evo_gate, list(range(n)))
    U_trot = Operator(qc_test).data           # (2**n,2**n) unitary from synthesized circuit

    # exact unitary:
    U_exact = expm(-1j * H_slice_dense * DELTA_T)

    print("|| U_trot - U_exact ||_inf =", np.max(np.abs(U_trot - U_exact)))
    print("|| U_trot - U_exact ||_2  =", np.linalg.norm(U_trot - U_exact))

    H_slice_opflow = (A * build_HD(n)) + (B * build_HP_from_sample(x_sample, Jij))
    evo_gate = PauliEvolutionGate(H_slice_opflow, time=DELTA_T,
                                synthesis=SuzukiTrotter(order=ORDER, reps=REPS))
    qc_test = QuantumCircuit(n)
    qc_test.append(evo_gate, list(range(n)))

    # 2) Transpile to basis gates to force use of the Suzuki-Trotter decomposition
    #    Choose basis_gates typical for backends, e.g. ['u3','cx'] or ['rz','sx','cx'] depending on Qiskit
    from qiskit import transpile
    transpiled = transpile(qc_test, basis_gates=['u3','cx'], optimization_level=0)

    # 3) Get unitary of the transpiled circuit (this reflects the trotter decomposition implemented)
    U_decomposed = Operator(transpiled).data

    # 4) The exact exponential
    U_exact = expm(-1j * H_slice_dense * DELTA_T)

    print("transpiled circuit depth:", transpiled.depth())
    print("|| U_decomposed - U_exact ||_inf =", np.max(np.abs(U_decomposed - U_exact)))
    print("|| U_decomposed - U_exact ||_2  =", np.linalg.norm(U_decomposed - U_exact))


|| HP_sparse - HP_dense ||_inf = 0.0
|| HD_sparse - HD_dense ||_inf = 0.0
|| H_slice_dense - H_slice_from_sparse ||_inf = 0.0
|| U_trot - U_exact ||_inf = 4.440892098501762e-16
|| U_trot - U_exact ||_2  = 1.34148758474959e-15


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


transpiled circuit depth: 2150
|| U_decomposed - U_exact ||_inf = 9.995416506238808e-15
|| U_decomposed - U_exact ||_2  = 3.754900131583076e-14
|| HP_sparse - HP_dense ||_inf = 8.881784197001252e-16
|| HD_sparse - HD_dense ||_inf = 0.0
|| H_slice_dense - H_slice_from_sparse ||_inf = 3.552713678800501e-15
|| U_trot - U_exact ||_inf = 4.440892098500628e-16
|| U_trot - U_exact ||_2  = 1.200889812746441e-15


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


transpiled circuit depth: 2150
|| U_decomposed - U_exact ||_inf = 1.0325074129014043e-14
|| U_decomposed - U_exact ||_2  = 3.415988644809347e-14
|| HP_sparse - HP_dense ||_inf = 0.0
|| HD_sparse - HD_dense ||_inf = 0.0
|| H_slice_dense - H_slice_from_sparse ||_inf = 0.0
|| U_trot - U_exact ||_inf = 4.440892098500752e-16
|| U_trot - U_exact ||_2  = 1.246222254317074e-15


...

transpiled circuit depth: 2150
|| U_decomposed - U_exact ||_inf = 1.2913030357952168e-14
|| U_decomposed - U_exact ||_2  = 6.067258734105144e-14
|| HP_sparse - HP_dense ||_inf = 0.0
|| HD_sparse - HD_dense ||_inf = 0.0
|| H_slice_dense - H_slice_from_sparse ||_inf = 0.0
|| U_trot - U_exact ||_inf = 4.440892098501762e-16
|| U_trot - U_exact ||_2  = 1.3089336417585332e-15


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


transpiled circuit depth: 2150
|| U_decomposed - U_exact ||_inf = 1.1892791543328825e-14
|| U_decomposed - U_exact ||_2  = 1.8345339328930762e-14
|| HP_sparse - HP_dense ||_inf = 3.552713678800501e-15
|| HD_sparse - HD_dense ||_inf = 0.0
|| H_slice_dense - H_slice_from_sparse ||_inf = 1.0658141036401503e-14
|| U_trot - U_exact ||_inf = 4.440892098503782e-16
|| U_trot - U_exact ||_2  = 1.2362920382607649e-15


c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_dsolve\linsolve.py:597: SparseEfficiencyWarning: splu converted its input to CSC format
  return splu(A).solve
c:\Users\jiriv\AppData\Local\Programs\Python\Python311\Lib\site-packages\scipy\sparse\linalg\_matfuncs.py:707: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  return spsolve(Q, P)


transpiled circuit depth: 2150
|| U_decomposed - U_exact ||_inf = 1.0214051826551705e-14
|| U_decomposed - U_exact ||_2  = 3.372959141096066e-14


In [26]:
T = 30e-9
REPS = 100
M = 3
DELTA_T = T / M

U_trotter_1st = expm(-1j * A * HD_dense * DELTA_T) @ expm(-1j * B * HP_dense * DELTA_T)

# Second-order symmetric Trotter (Strang) single-slice approx:
U_strang = (
    expm(-1j * (A/2) * HD_dense * DELTA_T) @
    expm(-1j * B * HP_dense * DELTA_T) @
    expm(-1j * (A/2) * HD_dense * DELTA_T)
)

U_exact = expm(-1j * H_slice_dense * T)

print("|| U_trotter_1st - U_exact ||_2 =", np.linalg.norm(U_trotter_1st**REPS - U_exact))
print("|| U_strang      - U_exact ||_2 =", np.linalg.norm(U_strang**REPS - U_exact))

|| U_trotter_1st - U_exact ||_2 = 9.979658652122076e-05
|| U_strang      - U_exact ||_2 = 9.97965865212205e-05
